<a href="https://colab.research.google.com/github/krishnajakodali/ml_lab3/blob/main/lab3_pruning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Basic Setup**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.utils.prune as prune
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import easydict
from torchsummary import summary
from google.colab import drive

# argument parser
import easydict

args = easydict.EasyDict({
        "batch_size": 32,
        "epochs": 10,
        "lr": 0.01,
})
# Hyper Parameters
input_size = 784
num_classes = 10
num_epochs = args.epochs
batch_size = args.batch_size
learning_rate = args.lr

# MNIST Dataset (Images and Labels)
train_set = dsets.FashionMNIST(
    root = './data/FashionMNIST',
    train = True,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor()
    ])
)
test_set = dsets.FashionMNIST(
    root = './data/FashionMNIST',
    train = False,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor()
    ])
)


# Dataset Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset = train_set,
        batch_size = batch_size,
        shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_set,
        batch_size = batch_size,
        shuffle = False)

100%|██████████| 26.4M/26.4M [00:02<00:00, 12.3MB/s]


Extracting ./data/FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 208kB/s]


Extracting ./data/FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 3.85MB/s]


Extracting ./data/FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 23.8MB/s]

Extracting ./data/FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/FashionMNIST/raw



# **Neural Net**

# Original net

In [ ]:
class MyConvNet(nn.Module):
    def __init__(self, args):
        super(MyConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.act1  = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.act2  = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.lin2  = nn.Linear(7*7*32, 10, bias=False)

    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.act2(x)
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = self.lin2(x)
        return x

model = MyConvNet(args)
model = model.cuda()

criterion = nn.CrossEntropyLoss()
criterion=criterion.cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 5e-4, momentum=0.9)

# Pruned Net

In [ ]:
class MyConvNet_pruned(nn.Module):
    def __init__(self, args):
        numChannels_conv1_output = 12
        numChannels_conv2_output = 15

        super(MyConvNet_pruned, self).__init__()
        self.conv1 = nn.Conv2d(1, numChannels_conv1_output, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.act1  = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(numChannels_conv1_output, numChannels_conv2_output, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.act2  = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.lin2  = nn.Linear(7*7*numChannels_conv2_output, 10, bias=False)

    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.act2(x)
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = self.lin2(x)
        return x

model_pruned = MyConvNet_pruned(args)
model_pruned = model_pruned.cuda()

In [ ]:
print("Initial model summary")
summary(model, (1, 28, 28))

print("Pruned model summary")
summary(model_pruned, (1, 28, 28))

Initial model summary
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             144
              ReLU-2           [-1, 16, 28, 28]               0
         MaxPool2d-3           [-1, 16, 14, 14]               0
            Conv2d-4           [-1, 32, 14, 14]           4,608
              ReLU-5           [-1, 32, 14, 14]               0
         MaxPool2d-6             [-1, 32, 7, 7]               0
            Linear-7                   [-1, 10]          15,680
Total params: 20,432
Trainable params: 20,432
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.32
Params size (MB): 0.08
Estimated Total Size (MB): 0.40
----------------------------------------------------------------
Pruned model summary
----------------------------------------------------------------


# **Functions**

In [ ]:
def get_modelAccuracy(dataset_loader):
    correct = 0
    total = 0

    for images, labels in dataset_loader:
        images = images.cuda()
        labels = labels.cuda()

        outputs = load_model(images)

        loss = criterion(outputs, labels)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    return (100 * correct / total)


def loadModel():
    drive.mount('/content/drive')

    !cp "/content/drive/My Drive/saved_myconvnet_lab4.pt" "./pretrainedModel"

    drive.flush_and_unmount()

    global load_model
    load_model = MyConvNet(args)
    load_model.load_state_dict(torch.load('./pretrainedModel'))

    load_model = load_model.cuda()
    load_model.eval()


def pruneModel(module, dim, amount):
    # dim = 0 --> channel pruning
    # dim = 2 --> filter row pruning
    # dim = 3 --> filter column pruning

    prune.ln_structured(module, name="weight", amount=amount,  n=1, dim=dim)
    #print("Post pruning")
    #print(list(module.named_parameters()))
    #print(module.weight)

def print_testAccuracy(test_loader):
    accuracy = get_modelAccuracy(test_loader).data.item()
    print('Accuracy for test images: % d %%' % accuracy)

# **Training**

In [ ]:
print("---Training started")
# Training the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.cuda()
        labels = Variable(labels).cuda()

        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        L1norm = model.parameters()
        arr = []
        for name,param in model.named_parameters():
          if 'weight' in name.split('.'):
            arr.append(param)
        L1loss = 0
        for Losstmp in arr:
          L1loss = L1loss+Losstmp.abs().mean()

        loss.backward()
        optimizer.step()

        if (i + 1) % 600 == 0:
            print('Epoch: [% d/% d], Step: [% d/% d], Loss: %.4f'
                    % (epoch + 1, num_epochs, i + 1,
                       len(train_set) // batch_size, loss.data.item()))

correct = 0
total = 0
model.eval()
for images, labels in test_loader:
    images = images.cuda()
    labels = labels.cuda()
    outputs = model(images)
    testloss = criterion(outputs, labels)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy for test images: % d %%' % (100 * correct / total))

---Training started
Epoch: [ 1/ 10], Step: [ 600/ 1875], Loss: 0.5599
Epoch: [ 1/ 10], Step: [ 1200/ 1875], Loss: 0.2484
Epoch: [ 1/ 10], Step: [ 1800/ 1875], Loss: 0.3158
Epoch: [ 2/ 10], Step: [ 600/ 1875], Loss: 0.5228
Epoch: [ 2/ 10], Step: [ 1200/ 1875], Loss: 0.2680
Epoch: [ 2/ 10], Step: [ 1800/ 1875], Loss: 0.4812
Epoch: [ 3/ 10], Step: [ 600/ 1875], Loss: 0.2206
Epoch: [ 3/ 10], Step: [ 1200/ 1875], Loss: 0.3994
Epoch: [ 3/ 10], Step: [ 1800/ 1875], Loss: 0.1739
Epoch: [ 4/ 10], Step: [ 600/ 1875], Loss: 0.2520
Epoch: [ 4/ 10], Step: [ 1200/ 1875], Loss: 0.4312
Epoch: [ 4/ 10], Step: [ 1800/ 1875], Loss: 0.2988
Epoch: [ 5/ 10], Step: [ 600/ 1875], Loss: 0.2379
Epoch: [ 5/ 10], Step: [ 1200/ 1875], Loss: 0.2737
Epoch: [ 5/ 10], Step: [ 1800/ 1875], Loss: 0.1225
Epoch: [ 6/ 10], Step: [ 600/ 1875], Loss: 0.2011
Epoch: [ 6/ 10], Step: [ 1200/ 1875], Loss: 0.3740
Epoch: [ 6/ 10], Step: [ 1800/ 1875], Loss: 0.1236
Epoch: [ 7/ 10], Step: [ 600/ 1875], Loss: 0.2469
Epoch: [ 7/ 10], S

# **Save Model**

In [ ]:
PATH = "./saved_myconvnet_lab4.pt"
torch.save(model.state_dict(), PATH)

from google.colab import drive
drive.mount('/content/drive')

!cp "./saved_myconvnet_lab4.pt" "/content/drive/My Drive"

drive.flush_and_unmount()

# **Load Model**

In [ ]:
from google.colab import drive

load_model = None

In [ ]:
# Validate pruning implementation
loadModel()
module = load_model.conv1
weights = module.weight.data
size = module.weight.size()
dims = [i for i in range(len(size))]

sum_index = {}
for i0 in range(size[3]):
    innerSum = 0
    for i1 in range(size[1]):
        for i2 in range(size[0]):
            for i3 in range(size[2]):
                innerSum += abs(weights[i2][i1][i3][i0].item())

    #print(i0, ": ", innerSum)
    sum_index[innerSum] = i0

sortedSums = sorted(sum_index.keys())
print("Total #sums = ", len(sortedSums))
for sum in sortedSums:
    index = sum_index[sum]
    print(index, " : ", sum)


# **Test**

In [ ]:
accuracy = get_modelAccuracy(test_loader).data.item()
print('Accuracy for test images: % d %%' % accuracy)
#summary(model, (1, 28, 28))

Accuracy for test images:  90 %


# **Prune**

## Testbench

In [ ]:
testVectorsConv1_dim_amount = [
    [0, 1/12],
    [2, 1/3],
    [3, 1/3]
]

testVectorsConv2_dim_amount = [
    [0, 1/15],
    [2, 1/3],
    [3, 1/3]
]

testVectorsOfLayer = {
    "conv1" : testVectorsConv1_dim_amount,
    "conv2" : testVectorsConv2_dim_amount
}

for layer in ["conv2", "conv1"]:
    print("Pruning layer: ", layer)

    for dim, amount in testVectorsOfLayer[layer]:
        loadModel()
        #pruneModel(module=load_model.conv2, dim=0, amount=5/32)
        #pruneModel(module=load_model.conv1, dim=0, amount=1/16)
        #pruneModel(module=load_model.conv2, dim=0, amount=12/32)
        pruneModel(module=load_model.conv1, dim=0, amount=4/16)
        pruneModel(module=load_model.conv2, dim=0, amount=17/32)
        accuracy_baseline = get_modelAccuracy(test_loader).data.item()

        if layer == "conv1":
            module = load_model.conv1
        elif layer == "conv2":
            module = load_model.conv2

        pruneModel(module=module, dim=dim, amount=amount)
        print("Pruned dim = ", dim, ", amount = ", amount)
        accuracy_pruned = get_modelAccuracy(test_loader).data.item()
        print("Accuracy for test images: %d%% --> %d%%" % (accuracy_baseline, accuracy_pruned))

Pruning layer:  conv2
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-4-30868408b8dc>:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  load_model.load_state_dict(torch.load('./pretrainedModel'))


Pruned dim =  0 , amount =  0.06666666666666667
Accuracy for test images: 86% --> 85%
Mounted at /content/drive
Pruned dim =  2 , amount =  0.3333333333333333
Accuracy for test images: 86% --> 80%
Mounted at /content/drive
Pruned dim =  3 , amount =  0.3333333333333333
Accuracy for test images: 86% --> 81%
Pruning layer:  conv1
Mounted at /content/drive
Pruned dim =  0 , amount =  0.08333333333333333
Accuracy for test images: 86% --> 83%
Mounted at /content/drive
Pruned dim =  2 , amount =  0.3333333333333333
Accuracy for test images: 86% --> 75%
Mounted at /content/drive
Pruned dim =  3 , amount =  0.3333333333333333
Accuracy for test images: 86% --> 65%


## Final Pruned Model

In [ ]:
loadModel()
accuracy_baseline = get_modelAccuracy(test_loader).data.item()
print("Initial model summary")
summary(load_model, (1, 28, 28))

pruneModel(module=load_model.conv1, dim=0, amount=4/16)
pruneModel(module=load_model.conv2, dim=0, amount=17/32)
accuracy_pruned = get_modelAccuracy(test_loader).data.item()
print("Accuracy for test images: %d%% --> %d%%" % (accuracy_baseline, accuracy_pruned))

prune.remove(module=load_model.conv1, name='weight')
prune.remove(module=load_model.conv2, name='weight')
print("Pruned model summary")
summary(load_model, (1, 28, 28))


Mounted at /content/drive


<ipython-input-4-1f8a9b2ac6e7>:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  load_model.load_state_dict(torch.load('./pretrainedModel'))


Initial model summary
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             144
              ReLU-2           [-1, 16, 28, 28]               0
         MaxPool2d-3           [-1, 16, 14, 14]               0
            Conv2d-4           [-1, 32, 14, 14]           4,608
              ReLU-5           [-1, 32, 14, 14]               0
         MaxPool2d-6             [-1, 32, 7, 7]               0
            Linear-7                   [-1, 10]          15,680
Total params: 20,432
Trainable params: 20,432
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.32
Params size (MB): 0.08
Estimated Total Size (MB): 0.40
----------------------------------------------------------------
Accuracy for test images: 90% --> 86%
Pruned model summary
---------------------------